In [2]:
from tavily import TavilyClient # For our search tool (Just an example tool)
from dotenv import load_dotenv # You should have your own .env file with required API keys
import os
from IPython.display import display, Markdown

In [3]:
# Make sure to define your own API keys in your .env file
TAVILY_API_KEY = "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"
AZURE_OPENAI_ENDPOINT="https://ue2coai4l3aoa02.openai.azure.com/"
AZURE_OPENAI_API_KEY="52220aca3ae5498fbe5ad757f139e6c4"
AZURE_OPENAI_VERSION="2025-01-01-preview"
# models available are "gpt-4o", "gpt-4o-mini", "gpt-4o-mini-audio-preview", and "text-embedding-3-small"

DALLE_AZURE_OPENAI_ENDPOINT="https://swccaoi0dwaoa01.openai.azure.com/"
DALLE_AZURE_OPENAI_API_KEY="4d55ac062cbb46e7a35c3e48bf1e8d37"
# model is "dall-e-3", api version date should be the same

In [4]:
from openai import AzureOpenAI

In [5]:
client = AzureOpenAI(
  azure_endpoint = AZURE_OPENAI_ENDPOINT, 
  api_key=AZURE_OPENAI_API_KEY,  
  api_version=AZURE_OPENAI_VERSION
)

response = client.chat.completions.create(
    model="gpt-4o", # Available models include "gpt-4o", "gpt-4o-mini", and "text-embedding-3-small"
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Does Azure OpenAI support customer managed keys?"},
        {"role": "assistant", "content": "Yes, customer managed keys are supported by Azure OpenAI."},
        {"role": "user", "content": "Do other Azure AI services support this too? Be concise."}
    ]
)

print(response.choices[0].message.content)

Yes, other Azure AI services, such as Azure Cognitive Services and Azure Machine Learning, also support customer-managed keys for enhanced data security.


In [6]:
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

response = tavily_client.qna_search("What is agentic AI?")

# Nicely prints out the final Markdown text
display(Markdown(response.replace("```", "")))

Agentic AI refers to a type of artificial intelligence that operates autonomously, making decisions, taking actions, and learning to complete complex tasks with minimal human intervention. It combines various forms of AI, including large language models and machine learning, to create intelligent systems capable of understanding context, adapting to new information, and collaborating with humans to solve intricate challenges. Agentic AI systems can analyze data, set goals, and manage workflows independently, thereby enhancing productivity, driving innovation, and unlocking new revenue streams for businesses.

In [7]:
# AutoGen Imports
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient # Azure OpenAI API
from autogen_agentchat.agents import AssistantAgent # Agent
from autogen_agentchat.agents import UserProxyAgent # Agent
from autogen_agentchat.conditions import TextMentionTermination # Termination condition
from autogen_agentchat.teams import RoundRobinGroupChat # Agent design pattern
from autogen_agentchat.ui import Console # To view agent outputs

def search_tool(query: str) -> str:
    answer = tavily_client.search(query=query)
    return answer

In [8]:
client = AzureOpenAIChatCompletionClient(
    model="gpt-4o",
    api_version=AZURE_OPENAI_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    temperature=0,
    model_capabilities={
        "vision": False,
        "function_calling": True,
        "json_output": False,
    },
)

In [30]:
# from autogen import AssistantAgent, UserProxyAgent

# Define the Market Research Orchestrator
orchestrator = UserProxyAgent(
    name="market_research_orchestrator",
    # model_client=client,
    description="Manages and coordinates market research tasks."
)

# Research Agent - Guides the conversation and compiles the research
research_agent = AssistantAgent(
    name="research_agent",
    model_client=client,
    description="Ensures proper market research is conducted for analysis.",
    system_message="""
        You are a research agent responsible for guiding market research. 
        Ensure the following aspects are covered:
        1. Competitor Analysis
        2. Industry Trends
        3. Regulatory Considerations
        4. Technological Advancements
        5. Consumer Behavior
        6. SWOT Analysis
        Collaborate with other agents and ensure all necessary information is gathered.
    """
)

# Competitor Research Agent
competitor_agent = AssistantAgent(
    name="competitor_research_agent",
    model_client=client,
    description="Finds competitor insights.",
    system_message="""
        You are a competitor research agent. 
        Your task is to analyze key competitors in this market, their strengths, and market positioning.
        Provide key insights into the competitive landscape.
    """,
    tools=[search_tool],
    reflect_on_tool_use=True,
)

# Industry Trends Research Agent
industry_agent = AssistantAgent(
    name="industry_trends_agent",
    model_client=client,
    description="Analyzes industry trends.",
    system_message="""
        You are an industry trends research agent.
        Your task is to provide an analysis of current industry trends, future outlook, and potential disruptions.
        Generate relevant graphs or data-driven insights.
    """,
    tools=[search_tool],
    reflect_on_tool_use=True,
)

# Regulatory Compliance Agent
regulation_agent = AssistantAgent(
    name="regulation_research_agent",
    model_client=client,
    description="Ensures regulatory compliance is analyzed.",
    system_message="""
        You are a regulatory research agent.
        Your task is to analyze key regulations affecting this market and compliance considerations.
        Ensure legal risks and guidelines are covered.
    """,
    tools=[search_tool],
    reflect_on_tool_use=True,
)

# Technology Research Agent
technology_agent = AssistantAgent(
    name="technology_research_agent",
    model_client=client,
    description="Researches emerging technologies.",
    system_message="""
        You are a technology research agent.
        Your task is to identify and analyze emerging technologies that influence this market.
        Provide details on technological shifts and industry impact.
    """,
    tools=[search_tool],
    reflect_on_tool_use=True,
)

# Consumer Behavior Research Agent
consumer_agent = AssistantAgent(
    name="consumer_research_agent",
    model_client=client,
    description="Analyzes consumer preferences and behaviors.",
    system_message="""
        You are a consumer behavior research agent.
        Your task is to analyze consumer behavior, preferences, and trends in this market.
        Ensure demographic and psychographic insights are provided.
    """,
    tools=[search_tool],
    reflect_on_tool_use=True,
)

# SWOT Analysis Agent
swot_agent = AssistantAgent(
    name="swot_analysis_agent",
    model_client=client,
    description="Conducts SWOT analysis.",
    system_message="""
        You are a SWOT analysis agent.
        Your task is to perform a SWOT (Strengths, Weaknesses, Opportunities, Threats) analysis for this market.
        Ensure a balanced evaluation of internal and external factors.
    """,
    tools=[search_tool],
    reflect_on_tool_use=True,
)

# Report Generation Agent
report_agent = AssistantAgent(
    name="report_generator_agent",
    model_client=client,
    description="Compiles research findings into a structured report.",
    system_message="""
    Summarize all findings into a structured market research report. END MESSAGE WITH: 'TERMINATE'."
    """,
    tools=[search_tool],
    reflect_on_tool_use=True,

)


In [31]:
import asyncio
from autogen_agentchat.ui import Console

In [35]:
from autogen_agentchat.teams import RoundRobinGroupChat

# Define the termination condition
termination = TextMentionTermination("TERMINATE")

# Define agents in a round-robin group
agent_group = RoundRobinGroupChat([
        competitor_agent,
        industry_agent,
        regulation_agent,
        technology_agent,
        consumer_agent,
        swot_agent,
        research_agent,
        report_agent
    ],termination_condition=termination 
)
async def execute_market_research(topic):
    """Executes market research using Round-Robin agent chat asynchronously."""
    
    print(f"🧠 Running market research for: {topic}")

    # Start the round-robin agent chat
    task_result = await Console(agent_group.run_stream(
        task=f"Conduct market research on {topic} and compile insights."
    ))
    research_results = task_result 
    return research_results


In [36]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader
import textwrap
import matplotlib.pyplot as plt

def generate_marketing_report(topic, research_results):
    pdf_filename = f"{topic}_market_research.pdf"
    c = canvas.Canvas(pdf_filename, pagesize=letter)
    width, height = letter

    # Title
    c.setFont("Helvetica-Bold", 16)
    c.drawString(100, height - 50, f"Market Research Report: {topic}")

    # Add Research Insights
    y_position = height - 80
    c.setFont("Helvetica", 10)

    for agent_name, response in research_results.items():
        if y_position < 100:  # Prevents overflow
            c.showPage()
            y_position = height - 50
        
        c.setFont("Helvetica-Bold", 12)
        c.drawString(100, y_position, f"{agent_name.replace('_', ' ')}:")
        y_position -= 20
        wrapped_text = textwrap.fill(response, width=90)
        for line in wrapped_text.split("\n"):
            c.drawString(100, y_position, line)
            y_position -= 15

    # Save PDF
    c.save()
    return pdf_filename


In [ ]:
topic = "Electric Vehicles"
research_results = await execute_market_research(topic)

print(f" Research Completed! Results: {research_results}")

# # Generate the Report
# pdf_report = generate_marketing_report(topic, research_results)

# print(f" Market Research Report Generated: {pdf_report}")

🧠 Running market research for: Electric Vehicles
---------- user ----------
Conduct market research on Electric Vehicles and compile insights.


---------- competitor_research_agent ----------
### Comprehensive Market Research Insights on Electric Vehicles (EVs)

#### **Key Competitors in the Electric Vehicle Market**
1. **Tesla**: 
   - Global leader in EVs, focusing exclusively on battery-electric vehicles (BEVs).
   - Known for innovation in battery technology (e.g., 4680 battery) and vertical integration.
   - Holds the largest global EV market share at 19.9% (2023).

2. **BYD**:
   - Dominates the Chinese market and is expanding globally.
   - Offers both plug-in hybrid electric vehicles (PHEVs) and BEVs.
   - Blade Battery technology emphasizes safety and energy density.
   - Outpaced Tesla in 2023 with 3.02 million new energy vehicle sales.

3. **Volkswagen**:
   - Legacy automaker transitioning aggressively into EVs.
   - Focuses on mass-market adoption with scalable EV platforms.
   - Competes in both premium and affordable EV segments.

4. **Ford**:
   - Strong presence in electric SUVs and trucks, such as the Mustang